In [1]:
import fla as a
import pickle as pkl
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from importlib import reload
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score

data prep

In [4]:
with open("MM_Attention/tadpole/X.pkl/tadpole/X.pkl", "rb") as file:
    X = pkl.load(file)
early = 0
with open("MM_Attention/tadpole/y.pkl", "rb") as file:
    if early:
        y = pkl.load(file)[:,1:]
    else:
        y = pkl.load(file)[:,:-1]

FileNotFoundError: [Errno 2] No such file or directory: 'MM_Attention/tadpole/X.pkl/tadpole/X.pkl'

In [60]:
X = X.reshape(X.shape[0]*X.shape[1], X.shape[2])
y = y.flatten()
weight = torch.tensor(compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y), dtype=torch.float32)

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, stratify=y_train, test_size=0.1, random_state=42)

build model

In [122]:
reload(a)

<module 'main' from '/Users/susmaa01/Documents/MM_Attention/main.py'>

In [123]:
train_dataset = a.npDataset(X_train,y_train)
test_dataset = a.npDataset(X_test,y_test)
val_dataset = a.npDataset(X_val,y_val)
batch_size = 100

train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [124]:
hidden_dims = [160,40,10]
attn_heads = 0
model = a.UTAMLP(input_dim=337, attn_heads=attn_heads, hidden_dims=hidden_dims, output_dim=3, activation=nn.ReLU())

In [125]:
criterion = nn.CrossEntropyLoss(weight=weight)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
metric = a.MacroF1Score(num_classes=3)

train


In [126]:
num_epochs = 50
best_val_loss = float('inf')
best_model = None
patience = 10
early_stop_counter = 0
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    val_losses = []
    for inputs, labels in val_loader:
        with torch.no_grad():
            outputs = model(inputs)
            val_loss = criterion(outputs, labels)
            val_losses.append(val_loss.item())
    
    avg_val_loss = np.mean(val_losses)
    print(f'Epoch {epoch+1}, Validation Loss: {avg_val_loss:.4f}')
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model = model.state_dict()
        early_stop_counter = 0
    else:
        early_stop_counter += 1
    
    if early_stop_counter >= patience:
        print(f'Early stopping after epoch {epoch+1} with validation loss {best_val_loss:.4f}')
        break

model.load_state_dict(best_model)
model.eval()

Epoch 1, Validation Loss: 0.7419
Epoch 2, Validation Loss: 0.5934
Epoch 3, Validation Loss: 0.4548
Epoch 4, Validation Loss: 0.3967
Epoch 5, Validation Loss: 0.4412
Epoch 6, Validation Loss: 0.3669
Epoch 7, Validation Loss: 0.3264
Epoch 8, Validation Loss: 0.3368
Epoch 9, Validation Loss: 0.3349
Epoch 10, Validation Loss: 0.3158
Epoch 11, Validation Loss: 0.3152
Epoch 12, Validation Loss: 0.3266
Epoch 13, Validation Loss: 0.3158
Epoch 14, Validation Loss: 0.3272
Epoch 15, Validation Loss: 0.3757
Epoch 16, Validation Loss: 0.2866
Epoch 17, Validation Loss: 0.3081
Epoch 18, Validation Loss: 0.3800
Epoch 19, Validation Loss: 0.3638
Epoch 20, Validation Loss: 0.3631
Epoch 21, Validation Loss: 0.4641
Epoch 22, Validation Loss: 0.4258
Epoch 23, Validation Loss: 0.4560
Epoch 24, Validation Loss: 0.4089
Epoch 25, Validation Loss: 0.3963
Epoch 26, Validation Loss: 0.4275
Early stopping after epoch 26 with validation loss 0.2866


UTAMLP(
  (utab_attn): UTabAttn(
    (weights): ParameterDict()
    (biases): ParameterDict()
  )
  (attn_norm): LayerNorm((337,), eps=1e-05, elementwise_affine=True)
  (linears): ModuleList(
    (0): Linear(in_features=337, out_features=160, bias=True)
    (1): Linear(in_features=160, out_features=40, bias=True)
    (2): Linear(in_features=40, out_features=10, bias=True)
  )
  (linear_norms): ModuleList(
    (0): LayerNorm((160,), eps=1e-05, elementwise_affine=True)
    (1): LayerNorm((40,), eps=1e-05, elementwise_affine=True)
    (2): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
  )
  (activation): ReLU()
  (output): Linear(in_features=10, out_features=3, bias=True)
)

eval

In [127]:
test_losses = []
test_predictions = []
test_true_labels = []

for inputs, labels in test_loader:
    with torch.no_grad():
        outputs = model(inputs)
        test_loss = criterion(outputs, labels)
        test_losses.append(test_loss.item())
        predictions = torch.argmax(outputs, dim=1)
        test_predictions.extend(predictions.cpu().numpy())
        test_true_labels.extend(labels.cpu().numpy())

avg_test_loss = np.mean(test_losses)
test_score = f1_score(test_true_labels, test_predictions, average='macro')
print(f'Test Loss: {avg_test_loss:.4f}, Test Score: {test_score:.4f}')

Test Loss: 0.4793, Test Score: 0.8387


In [129]:
best_score = None
no_attn_score = None
for h in range(15):
    hidden_dims = [160,40,10]
    attn_heads = h
    model = a.UTAMLP(input_dim=337, attn_heads=attn_heads, hidden_dims=hidden_dims, output_dim=3, activation=nn.ReLU())

    criterion = nn.CrossEntropyLoss(weight=weight)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    metric = a.MacroF1Score(num_classes=3)

    num_epochs = 50
    best_val_loss = float('inf')
    best_model = None
    patience = 10
    early_stop_counter = 0
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        val_losses = []
        for inputs, labels in val_loader:
            with torch.no_grad():
                outputs = model(inputs)
                val_loss = criterion(outputs, labels)
                val_losses.append(val_loss.item())
        
        avg_val_loss = np.mean(val_losses)
        print(f'Epoch {epoch+1}, Validation Loss: {avg_val_loss:.4f}')
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model = model.state_dict()
            early_stop_counter = 0
        else:
            early_stop_counter += 1
        
        if early_stop_counter >= patience:
            print(f'Early stopping after epoch {epoch+1} with validation loss {best_val_loss:.4f}')
            break

    model.load_state_dict(best_model)

    test_losses = []
    test_predictions = []
    test_true_labels = []

    for inputs, labels in test_loader:
        with torch.no_grad():
            outputs = model(inputs)
            test_loss = criterion(outputs, labels)
            test_losses.append(test_loss.item())
            predictions = torch.argmax(outputs, dim=1)
            test_predictions.extend(predictions.cpu().numpy())
            test_true_labels.extend(labels.cpu().numpy())

    avg_test_loss = np.mean(test_losses)
    test_score = f1_score(test_true_labels, test_predictions, average='macro')
    print(f'Test Loss: {avg_test_loss:.4f}, Test Score: {test_score:.4f}')
    if h==0:
        no_attn_score = test_score
        best_score = [0, test_score]
    else:
        if test_score > best_score[1]:
            best_score = [h, test_score]
    print(best_score)
no_attn_score, best_score

Epoch 1, Validation Loss: 0.7355
Epoch 2, Validation Loss: 0.5865
Epoch 3, Validation Loss: 0.4655
Epoch 4, Validation Loss: 0.4020
Epoch 5, Validation Loss: 0.3527
Epoch 6, Validation Loss: 0.3202
Epoch 7, Validation Loss: 0.3045
Epoch 8, Validation Loss: 0.3549
Epoch 9, Validation Loss: 0.3623
Epoch 10, Validation Loss: 0.3074
Epoch 11, Validation Loss: 0.4488
Epoch 12, Validation Loss: 0.4266
Epoch 13, Validation Loss: 0.3810
Epoch 14, Validation Loss: 0.3587
Epoch 15, Validation Loss: 0.3842
Epoch 16, Validation Loss: 0.5750
Epoch 17, Validation Loss: 0.4126
Early stopping after epoch 17 with validation loss 0.3045
Test Loss: 0.3866, Test Score: 0.8113
[0, 0.8112513424167015]
Epoch 1, Validation Loss: 0.6514
Epoch 2, Validation Loss: 0.5696
Epoch 3, Validation Loss: 0.4658
Epoch 4, Validation Loss: 0.4421
Epoch 5, Validation Loss: 0.3649
Epoch 6, Validation Loss: 0.3442
Epoch 7, Validation Loss: 0.3443
Epoch 8, Validation Loss: 0.3560
Epoch 9, Validation Loss: 0.3653
Epoch 10, Vali

KeyboardInterrupt: 